<Table border="1">
<tr><td>
<img alt="INTRO Seguridad y Privacidad Python" title="INTRO Seguridad y Privacidad Python" src="./UAM+EPS_L.png" width="800">
</td><td>
</table>

# Seguridad y Privacidad - Master Big Data y Data Science - 2021/2023

# Práctica 1: Tecnologías criptográficas para la protección de la seguridad y privacidad

En esta práctica continua se estudiarán y analizarán algunas de las funciones criptográficas que existen a través de Python, que nos pueden proporcionar seguridad y privacidad en los protocolos de seguridad.

El paquete de Python para las funciones criptográficas que se utilizará es La librería pycryptodome:

[pycryptodome](https://pypi.org/project/pycryptodome/): Es un paquete que tiene implementadas varias funciones criptográficas de cifrado y funciones hash que hemos visto en clase, y muchas más incluidos protocolos de firma. 


## 1) Cifrados simétricos: DES

In [2]:
# DES 64 bits clave y bloque, modo ECB,
# Recordar que es un cifrado de bloques.
# la librería  PycCypto soporta AES, ARC2, Blowfish, CAST, DES, DES3, IDEA, RC5
from Crypto.Cipher import DES
from Crypto import Random
import base64 

# versión de python
from platform import python_version
print("Versión de python:",python_version())

# versión de pycryptodome
import Crypto
print("Versión de pycryptodome:",Crypto.__version__)
del Crypto


key = b'11111111'         # para expresar que son bytes ya que la codificacion en python es diferente,
                          # por ejemplo el caracter S puede tener una codificación diferente en función del python y
                          # la plataforma en la que se ejecute.
cipher = DES.new(key, DES.MODE_ECB)
msg = b'22222222' # bloque de 8 bytes
print('Mensaje plano -->', msg)
# Cifrado
msgc = cipher.encrypt(msg)
print('Mensaje cifrado -->', msgc)
# transformar a hexadecimal por los caracteres no imprimibles
print('Mensaje cifrado -->', msgc.hex())
# Descifrado
msgd = cipher.decrypt(msgc)
print('Mensaje descifrado -->', msgd)

Versión de python: 3.8.12
Versión de pycryptodome: 3.14.1
Mensaje plano --> b'22222222'
Mensaje cifrado --> b"\xf9'\x14\xe8\x0e\x83.\xdc"
Mensaje cifrado --> f92714e80e832edc
Mensaje descifrado --> b'22222222'


In [16]:
# DES 64 bits clave y bloque, modo CFB
# El objeto de cifrador tiene un estado interno cuando se utiliza en el modos de en cadenado como CBC, 
# por lo que si intenta cifrar el mismo texto con el mismo cifrador una vez más, obtendrá resultados diferentes. 
# Por lo tanto, hay que tener cudado cuidado de crear un nuevo objeto cifrador para cualquier trabajo de cifrado / descifrado.
# También cuando desciframos tenemos que crear un nuevo objeto. Con este nuevo objeto de descifrado tenemos que utilizar el
# IV adecuado y normalmente se pasa en linea, poniendolo al principio del mensaje cifrado.
# 
from Crypto.Cipher import DES
from Crypto import Random

key = b'11111111'         # para expresar que son bytes ya que la codificacion en python es diferente,
                          # por ejemplo el caracter S puede tener una codificación diferente en función del python y
                          # la plataforma en la que se ejecute.
iv = Random.new().read(DES.block_size)
print('IV -->', iv.hex())
cipher1 = DES.new(key, DES.MODE_CFB, iv) # se crea un cifrador
msg = b'2222222233333333' # 2 bloques de 8 bytes
print('Mensaje plano -->', msg)
# Cifrado, Emisor
msgc = iv + cipher1.encrypt(msg) # El iv se añade al principio en el mensaje a enviar para luego al decifrar cogerlo
print('IV + Mensaje cifrado -->', msgc.hex())

####
####
# Ahora el Emisor envia el mensaje por la red al Receptor
print('-->')
print('-->')
print('Envio de mensaje cifrado por canal inseguro')
print('-->')
print('-->')
####
####

# Descifrado, Receptor
iv_enviado = msgc[:DES.block_size] # primero se saca el iv  del mensaje que estaba al principio
cipher2 = DES.new(key, DES.MODE_CFB, iv_enviado) # se crea un nuevo cifrador 
solo_mensaje=msgc[DES.block_size:]
print('Mensaje cifrado -->', solo_mensaje.hex())
msgd = cipher2.decrypt(solo_mensaje) # Deciframos el mensaje sin el iv
print('IV_enviado -->', iv_enviado.hex())
print('Mensaje descifrado -->', msgd)

IV --> 133e568fddf94185
Mensaje plano --> b'2222222233333333'
IV + Mensaje cifrado --> 133e568fddf941850938a867f22bc2c620f15a2d5dc8c9cd
-->
-->
Envio de mensaje cifrado por canal inseguro
-->
-->
Mensaje cifrado --> 0938a867f22bc2c620f15a2d5dc8c9cd
IV_enviado --> 133e568fddf94185
Mensaje descifrado --> b'2222222233333333'


## 2) Cifrados simétricos: AES

In [3]:
# AES 128 bit bloque y 128 bits clave, modo ECB
from Crypto.Cipher import AES
from Crypto import Random
# Definición de la función lambda que comvierte una cadena a hexadecimal.
HEXA= lambda xx: ':'.join(hex(ord(x))[2:] for x in xx)

key = b'1111111111111111' # para expresar que son bytes ya que la codificacion en python es diferente,
                          # por ejemplo el caracter S puede tener una codificación diferente en función del python y
                          # la plataforma en la que se ejecute.
cipher = AES.new(key, AES.MODE_ECB)
msg = b'2222222222222222' # Mensaje de 16 bytes
print('Mensaje plano -->', msg)
# Cifrado
msgc = cipher.encrypt(msg)
print('Mensaje cifrado -->', msgc)
# Descifrado
msgd = cipher.decrypt(msgc)
print('Mensaje descifrado -->', msgd)

Mensaje plano --> b'2222222222222222'
Mensaje cifrado --> b'2\x88\xa9\x84\xdd\x06\xffX\xe99`\xb1\xde\xc5\x0f\x1e'
Mensaje descifrado --> b'2222222222222222'


## Tu turno 1: Probar el AES con el modo de operación CFB. Probar el AES con tamaños de claves superiores a 128 en modo CFB. Probar el DES3 con el modo de operación CFB.

Para probar la operación CFB basta cambiar el atributo AES.MODE_ECB por AES.MODE_CFB. A contiuación se encripta y desencripta el mensaje propuesto en la práctica.

In [224]:
from Crypto.Cipher import AES
from Crypto import Random
# Definición de la función lambda que comvierte una cadena a hexadecimal.
HEXA= lambda xx: ':'.join(hex(ord(x))[2:] for x in xx)

key = b'1111111111111111' # para expresar que son bytes ya que la codificacion en python es diferente,
                          # por ejemplo el caracter S puede tener una codificación diferente en función del python y
                          # la plataforma en la que se ejecute.
cipher = AES.new(key, AES.MODE_CFB)
msg = b'2222222222222222' # Mensaje de 16 bytes
print('Mensaje plano -->', msg)
# Cifrado
msgc = cipher.encrypt(msg)
print('Mensaje cifrado -->', msgc)

Mensaje plano --> b'2222222222222222'
Mensaje cifrado --> b'{\xe0h\x16\x8b\xa9^\xb3\xf7^,\xe7zB\x02\xb0'


In [225]:
cipher2 = AES.new(key, AES.MODE_CFB)

In [226]:
# Descifrado
cipher2.decrypt(cipher.encrypt(msg))
print('Mensaje descifrado -->', cipher2.decrypt(cipher.encrypt(msg)))

Mensaje descifrado --> b'2222222222222222'


A continuación, se cambia el número bytes de la clave respecto al ejemplo propuesto a 24 (AES-196) o 36 (AES-256).

doc:https://pycryptodome.readthedocs.io/en/latest/src/cipher/aes.html#:~:text=AES%20(Advanced%20Encryption%20Standard)%20is,block%20size%20of%2016%20bytes.&text=The%20secret%20key%20to%20use,192%20or%20AES%2D256).

In [237]:
from Crypto.Cipher import AES
from Crypto import Random
# Definición de la función lambda que comvierte una cadena a hexadecimal.
HEXA= lambda xx: ':'.join(hex(ord(x))[2:] for x in xx)

key = b'1'*24 #32
cipher = AES.new(key, AES.MODE_CFB)
msg = b'2222222222222222' # Mensaje de 16 bytes
print('Mensaje plano -->', msg)
# Cifrado
msgc = cipher.encrypt(msg)
print('Mensaje cifrado -->', msgc)

Mensaje plano --> b'2222222222222222'
Mensaje cifrado --> b';\xb6SJ\nFf\x1d\xe0|\x98cy\xa6\xda\t'


In [238]:
cipher2 = AES.new(key, AES.MODE_CFB)

In [239]:
# Descifrado
cipher2.decrypt(cipher.encrypt(msg))
print('Mensaje descifrado -->', cipher2.decrypt(cipher.encrypt(msg)))

Mensaje descifrado --> b'2222222222222222'


A continuación se prueba el DESC3 reusando el sódigo propuesto.

In [242]:
from Crypto.Cipher import DES3
from Crypto import Random

key = b'11111111'         # para expresar que son bytes ya que la codificacion en python es diferente,
                          # por ejemplo el caracter S puede tener una codificación diferente en función del python y
                          # la plataforma en la que se ejecute.
iv = Random.new().read(DES3.block_size)
print('IV -->', iv.hex())
cipher1 = DES3.new(key, DES3.MODE_CFB, iv) # se crea un cifrador
msg = b'2222222233333333' # 2 bloques de 8 bytes
print('Mensaje plano -->', msg)
# Cifrado, Emisor
msgc = iv + cipher1.encrypt(msg) # El iv se añade al principio en el mensaje a enviar para luego al decifrar cogerlo
print('IV + Mensaje cifrado -->', msgc.hex())

####
####
# Ahora el Emisor envia el mensaje por la red al Receptor
print('-->')
print('-->')
print('Envio de mensaje cifrado por canal inseguro')
print('-->')
print('-->')
####
####

# Descifrado, Receptor
iv_enviado = msgc[:DES3.block_size] # primero se saca el iv  del mensaje que estaba al principio
cipher2 = DES3.new(key, DES.MODE_CFB, iv_enviado) # se crea un nuevo cifrador 
solo_mensaje=msgc[DES3.block_size:]
print('Mensaje cifrado -->', solo_mensaje.hex())
msgd = cipher2.decrypt(solo_mensaje) # Deciframos el mensaje sin el iv
print('IV_enviado -->', iv_enviado.hex())
print('Mensaje descifrado -->', msgd)

IV --> ab0d4d28e71d1f0e


ValueError: Not a valid TDES key

## 3) Cifrados de flujo: ARC4 (RC4)

In [4]:
# RC4, recordar que este algoritmo trabaja byte a byte, es decir el tamaño de bloque es un byte, y el modo es ECB
# # la librería  PycCypto soporta RC4, XOR
from Crypto.Cipher import ARC4
from Crypto.Hash import SHA
from Crypto import Random

msg=b'Hola, quiero que me cifres en flujo con rc4'
key = b'Very long and confidential key'
print('clave elegida por el Emisor y Receptor -->',"%s"  % key)
nonce = Random.new().read(16) # Nonce de 16 bytes
print('nonce generado -->',"%s"  % nonce.hex())
key_temporal = SHA.new(key+nonce).digest() # combina la clave y el nonce de manera compleja
cifrador1 = ARC4.new(key_temporal)
cipher_text = nonce + cifrador1.encrypt(msg)
print('Mensaje cifrado -->',"%s"  % cipher_text.hex())


####
####
# Ahora el Emisor envia el mensaje por la red al Receptor
print('-->')
print('-->')
print('Envio de mensaje cifrado por canal inseguro')
print('-->')
print('-->')
####
####


# el receptor conoce key = b'Very long and confidential key', ya que es un cifrado simétrico.
# solo le hace falta sacar el nonce del mensaje de 16 bbytes para sacar la key_temporal a través del hash sha

nonce_enviado = cipher_text[:16] # primero se saca el nonce del mensaje, i.e. los 16 primero bytes
print('nonce recibido, extraído del mensaje cifrado recibido -->',"%s"  % nonce_enviado.hex())
key_temporal = SHA.new(key+nonce_enviado).digest()
cifrador2 = ARC4.new(key_temporal)
plain_text=cifrador2.decrypt(cipher_text[16:]) # descifra solo el mensaje, i.e. a partir de lo 16 bytes de nonce
print('Mensaje descifrado -->', "%s"  % plain_text)

clave elegida por el Emisor y Receptor --> b'Very long and confidential key'
nonce generado --> dfdce12316aab085a4d9bd33d228abcf
Mensaje cifrado --> dfdce12316aab085a4d9bd33d228abcf861f3f6ae0d4babd0eb01ed952aa6ec80b00862aff8aa369b729852d197d64ff350f49743c06d73cab39ca
-->
-->
Envio de mensaje cifrado por canal inseguro
-->
-->
nonce recibido, extraído del mensaje cifrado recibido --> dfdce12316aab085a4d9bd33d228abcf
Mensaje descifrado --> b'Hola, quiero que me cifres en flujo con rc4'


## Tu turno 2: Explicar con detalle porque para el cifrado de flujo RC4, como mostramos en el código anterior, la clave de largo plazo se combina complejamente con un Nonce a través de una función Hash ¿Porqué no se utiliza solo la clave de largo plazo? ¿Porqué no se utiliza ninguna función de Padding? ¿Qué relación tiene RC4 con OTP?

## 4) Números grandes y primos

In [19]:
# Aquí vemos algún ejemplo de utilización de nuúmeros grandes y primos, que se puede necesitar parra criptografía públlica.
a=2071807432749382749858321984570321740321403217098750948590849058904285904285908490589048590485094809583409859034285392
b=221740321403217098750948590849058904285904285908490589048590485094809583409859034285392
c=a*b
print('a=', a)
print('b=', b)
print('c=a*b=', c)





a= 2071807432749382749858321984570321740321403217098750948590849058904285904285908490589048590485094809583409859034285392
b= 221740321403217098750948590849058904285904285908490589048590485094809583409859034285392
c=a*b= 459403246023422225709304167202868039658205344539028817028410035649877817008105003350087322134711902659544448208557981220575724658000452849045673521548530808629308809973244668816985865008426134944104593664


In [20]:
# Utiliza un algorimos de primalidad para obtener un primo de 4096 bits. Es un algoritmo costoso computacionalmente hablando.
from  Crypto.Util import number
p=number.getPrime(4096)
p

9769045455200428477606806638412137211795360735161411923952358143430020613343261183139502106479924772246443819584035844014769657598674263997579216808006852246501595127801425871927149409155031480701245482577730710826225131746072330017880727930578483646978225176190965323450589849700039823177027991960387132241312782502255804579764453688467845457185325686223680253940141530311302497358781066347840830978363817501798611877039186594251825494133101428473340144780643794782256917946248486768040385249875040908575940278695114961027112618672992510655042980978143779779641366581111122038013866554275548604702309313952005036827055217502024486651274900278988007108013795945544484825945172516856080156089705901031204529932616793823806168954337024855106927667761310915611390143863337818016482817252452142476824107988810969905306842579631499936604634930588106558692647786126775875783736211402489611676467677480463902663582296656205399260996862004140425201629910588109913111076645273985036702672572914767540168802739

## 5) Obtención de parejas de claves Privada-Pública para RSA y su cifrado

In [21]:
from  Crypto.Util import number
from Crypto.PublicKey import RSA # manejo de claves RSA
from Crypto.Cipher import PKCS1_OAEP # cifrador RSA
from Crypto import Random


generador_aleatorio = Random.new().read
numero_bits_clave=1024
key = RSA.generate(numero_bits_clave, generador_aleatorio) # la clave tiene toda la información que se necesita mas lo métodos asociados
print('KEY --> ' "%s"  % key)
print('n --> ' "%s"  % key.n)
print('e --> ' "%s"  % key.e)
print('d --> ' "%s"  % key.d)
print('p --> ' "%s"  % key.p)
print('q --> ' "%s"  % key.q)
print('n=p*q --> ' "%s"  % (key.p*key.q))
print('¿Es primo p?--> ' "%s"  % (number.isPrime(key.p)))
print('¿Es primo q?--> ' "%s"  % (number.isPrime(key.q)))

# Se pueden chequear las posibilidades de la clave generada para el objeto RSA:
print('¿Puede cifrar? --> ' "%s"  % key.can_encrypt())
print('¿Puede firmar? --> ' "%s"  % key.can_sign())
print('¿La clave privada está contenida ene el objeto? --> ' "%s"  % key.has_private())

# Por ejemplo para el estandar de firma digital DSA, cambian las cosas
from Crypto.Random import random
from Crypto.PublicKey import DSA
key_DSA = DSA.generate(1024)
print('¿Puede cifrar? --> ' "%s"  % key_DSA.can_encrypt())
print('¿Puede firmar? --> ' "%s"  % key_DSA.can_sign())
print('¿La clave privada está contenida ene el objeto? --> ' "%s"  % key_DSA.has_private())

# Volvemos al RSA y sacamos las claves de manera conveniente para cifrar y decifrar
public_key = key.publickey() # hay que generar la nueva clave pública, para publicarla en algún sitio y que el receptor la pueda leer 
print('public_key --> ' "%s"  % public_key)  # es un objeto compuesto por e y n que tiene asociado métodos para cifrar y descifrar
print('n --> ' "%s"  % key.n)
print('e --> ' "%s"  % key.e)
msg='Hola, quiero que me cifres con RSA' # mensaje a cifrar
# Importamos la clave pública para cifrar los datos
cifrar_rsa = PKCS1_OAEP.new(public_key)
# Importamos la clave privada para descifrar los datos
descifrar_rsa = PKCS1_OAEP.new(key)
# Ahora ciframos y desciframos
msgc = cifrar_rsa.encrypt(msg.encode()) # Sino se pone "b" en el mensaje, e deben codificar los datos a 'utf-8' ya que en En Python 3, 
                                        # no hay conversión implícita entre objetos Unicode (str) y objetos bytes.
print('msgc --> ' "%s"  % msgc) # Cifrado con la clave publica
print('msgc --> ' "%s"  % msgc.hex()) 
msg_descifrado = descifrar_rsa.decrypt(msgc) # Descifrado con la clave privada
#msg_descifrado = key.decrypt(msgc) # Descifrado con la clave privada
print('Mensaje decifrado --> ' "%s"  % msg_descifrado.decode())


KEY --> Private RSA key at 0x19D5E82B160
n --> 152957665405241898314482673652748158578526231702368131219480690378513394569202408607177026912685014287708786793367770981182131100820541266813899425505181495270352900172172050171092219241662533489914726608279573345974975015857551018088683098493615056443086220329477936798130419205274039421885056437501260596579
e --> 65537
d --> 40293843392111307405746160478475831725269178131826824563814705876677975808169354462343527795505598809377120551659335674574956168120546175456743009775153050910506857752781925780334696588209174181667815292080069945533758170237580884582780073707226318553680253297165364714940492133834329406713014302132377328833
p --> 11765445956732192385664990416478487342803736410142736946120578714932509123934704034542394029523836007666919902510241968804938746553669975770411708034861249
q --> 1300058374053551854082074775194783361413638170429623268230834416421014848857182036012083934102862187523096546603981903890390856169199432865419059095

## Tu turno 3: Utiliza RSA para cifrar un mensaje pero invirtiendo el procedimiento del protocolo: primero utiliza la clave privada y luego la clave pública. Prueba que recuperas el mensaje ¿Explica que diferencia hay con el procedimiento anterior?

## 6) Firma y verificacion de un mensaje con RSA

In [22]:
from Crypto.Hash import SHA512 # digest de 512

from Crypto.PublicKey import RSA
from Crypto import Random

generador_aleatorio = Random.new().read
numero_bits_clave=1024
key = RSA.generate(numero_bits_clave, generador_aleatorio) # la clave tiene toda la información que se necesita mas lo métodos asociados
msg=b'Hola, quiero firmar este mensaje con RSA' # mensaje a firmar
hash = SHA512.new(msg).digest()
print('Digest de SHA512--> ' "%s"  % (hash.hex()))
public_key = key.publickey() # hay que generar la nueva clave pública, para publicarla en algún sitio y que el receptor la pueda leer 
hash_entero = int.from_bytes(hash, byteorder='big') # transformo el digest a un número entero
firma = pow(hash_entero, key.d, key.n) # potenciación modular, una de las posibilidades de firma en RSA es cifrar el hash con la clave privada.
print('Firma--> ' "%s"  % firma)


####
####
# Ahora el Emisor envia el mensaje y su firma por la red al Receptor
print('-->')
print('-->')
print('Envio de mensaje y firma por canal inseguro (msg+firma)')
print('-->')
print('-->')
####
####

# Verificación de la firma
hash_receptor= SHA512.new(msg).digest()
hash_receptor_entero = int.from_bytes(hash, byteorder='big') # transformo el digest a un número entero
hash_firma = pow(firma, key.e, key.n)
print("Firma valida:", hash_receptor_entero == hash_firma) # Si el hash que calcula el receptor es igual al hash 
                                                           # contenido en la firma la verificación es correcta


# Aquí hemos implementado el protocolo de firma de la transparecia 120. Recordar que la función de cifrado y descifrado en RSA 
# es la potenciación modular: Clave pública PU = {e, n}, Clave privada PR = {d, n},
# y el Cifrado/Descifrado:
# Texto plano: M<n,   C = M^e mod n (función de potenciación modular)
# Texto cifrado: C,   M = C^d mod n (función de potenciación modular)

# PyCryptodome viene con varios estándares de firmas, como son PKCS#1 v1.5 (RSA), PKCS#1 PSS (RSA), 
# Digital Signature Algorithm (DSA and ECDSA). El estandar PKCS#1 v1.5, tiene una estructura muy similar al implementado arriba.
# Para más información ver el paquete de firmas de PyCryptodome: 
# https://pycryptodome.readthedocs.io/en/latest/src/signature/signature.html#



Digest de SHA512--> eeb99b864b3b52e7ebb665ea2b7c32cdec75d9f2edad3118e5e97c885db63e8de428899852a460754d90809abcee4c569a4e9a49e625a3007ec51e4227b79cde
Firma--> 75311040685164201056507085314057779784546820098877501708686383469882189767129621419161064998440506640387604609874383629180077028021054250221494848438125099082012378022885818040899352232523088014028810274418099158345351428191067726028717551597993013723331081697736066853921250265114115185661824716400632910348
-->
-->
Envio de mensaje y firma por canal inseguro (msg+firma)
-->
-->
Firma valida: True


## 7) Intercambio de Claves Diffie–Hellman

In [23]:
# función de exponenciación modular rápida
# Ejemplo 88^7=187 --> pow_mod(88,7,187)
def pow_mod(x,e,m):
    X = x
    E = e
    Y = 1
    while E > 0:
        if E % 2 == 0:
            X = (X * X) % m
            E = E/2
        else:
            Y = (X * Y) % m
            E = E - 1
    return Y
########
 

# Números públicos
primo_compartido=67
print('primo_compartido--> ' "%s"  % primo_compartido)
g=50
print('g--> ' "%s"  % g)
# Alice elige su secreto
secreto_Alice = 27
print('secreto_Alice--> ' "%s"  % secreto_Alice)
# Bob elige su secreto
secreto_Bob = 13
print('secreto_Bob--> ' "%s"  % secreto_Bob)

# Alice envía Bob A = g^a mod p
A = pow_mod(g,secreto_Alice,primo_compartido)
print('Alice envía Bob A = g^a mod p --> ' "%s"  % A)

# Bob envía Alice B = g^b mod p
B = pow_mod(g,secreto_Bob,primo_compartido)
print('Bob envía Alice B = g^b mod p --> ' "%s"  % B)

# Ahora cada uno de ellos calculan secretamente la clave para el cifrado

# Alice calcula la clave de cifrado: s = B^a mod p
K_Alice = pow_mod(B, secreto_Alice, primo_compartido)
print('K_Alice --> ' "%s"  % K_Alice)

# Bob calcula la clave de cifrado: s = A^b mod p
K_Bob = pow_mod(A, secreto_Bob, primo_compartido)
print('K_Bob --> ' "%s"  % K_Bob)



primo_compartido--> 67
g--> 50
secreto_Alice--> 27
secreto_Bob--> 13
Alice envía Bob A = g^a mod p --> 58
Bob envía Alice B = g^b mod p --> 61
K_Alice --> 53
K_Bob --> 53


## Tu turno 4: Explicar porque Alice y Bob tienen una clave secreta común, k=53, cuando no se han pasado explicitamente la clave k=53.

## 8) Funciones Hash

In [24]:
from Crypto.Hash import SHA256
from Crypto.Hash import SHA512


msg = b'Hola, quiero un hash de este mensaje' # mensaje para hash
d256 = SHA256.new(msg).hexdigest()
print('d256 --> ' "%s"  % d256)

# metiendo un espacio al final
msg = b'Hola, quiero un hash de este mensaje ' # mensaje para hash
d256_ = SHA256.new(msg).hexdigest()
print('d256 con un espacio al final --> ' "%s"  % d256_)

msg = b'Hola, quiero un hash de este mensaje' # mensaje para hash
d512 = SHA512.new(msg).hexdigest()
print('d512 --> ' "%s"  % d512)

# metiendo un espacio al final
msg = b'Hola, quiero un hash de este mensaje ' # mensaje para hash
d512_ = SHA512.new(msg).hexdigest()
print('d512 con un espacio al final --> ' "%s"  % d512_)

# Las funciones hash se puede utilizar para muchas cosas
# por ejemplo para passwords o para chequear la integridad de ficheros eu se descargan por la red.
def chequear_password(mi_password, hash_mi_password):
    return SHA512.new(mi_password).hexdigest() == hash_mi_password

msg = b'pepito' # supuesta password
d512 = SHA512.new(msg).hexdigest()
print('HASH de la password d512 --> ' "%s"  % d512) # hash de la passwordo que copio en el seguno chequeo

ok=chequear_password(b'pepito', 'e58ba2e1ed71816c0135750e0ed9d5cb195ee75e14c90c07b2025fe46d2d1185cdaeb1a7bf42cbaf2d61ed89e3394ee86d35b6f5c846e9e8b271d744eb472195')
print('OK --> ' "%s"  % ok)

ok=chequear_password(b'pepito', '7182aec1a70db349e8843575acbfc34877020381519f7821c03fdb837dac89cb7d7d830ff95771da682a039f992f7f5b465191e061e4d580dba58bf9a5bd1d45')
print('OK --> ' "%s"  % ok)


# para calcular la integridad de un fichero que te descargas por md5
# Cualquier modificación de un fichero se puede detectar con un hash
import os
from Crypto.Hash import MD5

def obtener_md5_fichero(nombre_fichero):
    hash_md5 = MD5.new()
    tamano_trozo_fichero = 8192
    with open(nombre_fichero, 'rb') as f:
        while True:
            trozo_fichero = f.read(tamano_trozo_fichero)
            if len(trozo_fichero) == 0:
                break
            hash_md5.update(trozo_fichero) # Al utilizar 'update' por trozos del fichero la memoria no se recarga
    return hash_md5.hexdigest()

h=obtener_md5_fichero('Logo-UAM.gif')
print('MD5 del fichero Logo-UAM.gif--> ' "%s"  % h)


d256 --> e5ae018c198ddb056d81e80b8f46e4466ce663213513a31dc4933b79e5748b10
d256 con un espacio al final --> 41f74ce2bef0fc6945a17fc8252cfd8a25ce3c3f0a557ac8ffa6915aba324531
d512 --> 0599286b29a02e1618bc69f97912e8aa8c3c484f4d3412c0fa3b5b1fe2211980f6e84940087ba864f9ea5601d21aacfadc6b7e3af1dc167b80a2990b52571b30
d512 con un espacio al final --> e58ba2e1ed71816c0135750e0ed9d5cb195ee75e14c90c07b2025fe46d2d1185cdaeb1a7bf42cbaf2d61ed89e3394ee86d35b6f5c846e9e8b271d744eb472195
HASH de la password d512 --> 7182aec1a70db349e8843575acbfc34877020381519f7821c03fdb837dac89cb7d7d830ff95771da682a039f992f7f5b465191e061e4d580dba58bf9a5bd1d45
OK --> False
OK --> True
MD5 del fichero Logo-UAM.gif--> 381eb171f5c7e0e7705a181c0cd8ef8e


## Tu turno 5: Implementa el esquema Firma Digital y Confidencialidad:
<Table border="1">
<tr><td>
<img alt="INTRO Seguridad y Privacidad Python" title="INTRO Seguridad y Privacidad Python" src="./FirmayConfidencialidad.png" width="1000">
</td></tr>
</table>

## Tu turno 6: Implementa este esquema de distribución de claves (OPCIONAL):
<Table border="1">
<tr><td>
<img alt="INTRO Seguridad y Privacidad Python" title="INTRO Seguridad y Privacidad Python" src="./KDC.png" width="1000">
</td></tr>
</table>